In [1]:
#r "nuget: FSharp.Collections.ParallelSeq" 

open FSharp.Collections.ParallelSeq
open System.Threading

let parseGrid lines =
    lines
    |> Array.map (
            Seq.map (
                fun c ->
                    match c with
                    | 'E' -> 25
                    | 'S' -> 0
                    | _ -> int c - int 'a'
            ) >> Array.ofSeq
        )

let positionOf ch lines =
    lines
    |> Array.indexed
    |> Array.pick (
        fun (r, l) ->
            l 
            |> Seq.tryFindIndex ((=) ch)
            |> Option.map (fun c -> (r,c))
        )

let startPos lines = lines |> positionOf 'S'
let endPos lines = lines |> positionOf 'E'

let possibleMoves (grid:int[][]) (r, c) =
    let curAlt = grid[r][c]
    let canClimb (r',c') = grid[r'][c'] - curAlt <= 1
    let h = grid.Length
    let w = grid[0].Length
    [
        if r > 0 then
            (r-1,c)
        if r < (h-1) then
            (r+1, c)
        if c > 0 then
            (r,c-1)
        if c < (w-1) then
            (r, c+1)
    ]
    |> List.filter canClimb

//let collectShorter (limit:int) (measure : 'u -> int) (f : int -> 't -> 'u list) (xs:'t list) : 'u list =
//    xs
//    |> List.scan (
//        fun (limit,_) x -> 
//            let fxs : 'u list = f limit x
//            let limit =
//                match fxs with
//                | [] -> limit
//                | fxs -> fxs |> List.map measure |> List.min |> min limit
//            (limit, fxs)
//        ) (limit,[])
//    |> List.map snd
//    |> List.concat

let walkPaths grid start dest =
    let limit = ref Int32.MaxValue
    let rec walk limit (pos, path) =
        if pos = dest then
            let length = Set.count path
            if length < !limit then Interlocked.Exchange(limit,length) |> ignore
            printfn "Found a result %d" length
            [ path ]
        else
            if Set.count path >= !limit then
                []
            else
                possibleMoves grid pos
                |> List.filter (fun pm -> not (path |> Set.contains pm))
                |> PSeq.map (fun pm -> pm, (Set.add pos path))
                |> PSeq.collect (walk limit)
                |> PSeq.toList
    walk limit (start, Set.empty)

Installed Packages FSharp.Collections.ParallelSeq, 1.2.0

In [ ]:
#r "nuget: FsUnit"
open FsUnitTyped

let testInput = 
    [|
        "Sabqponm"
        "abcryxxl"
        "accszExk"
        "acctuvwj"
        "abdefghi"
    |]

let testGrid = parseGrid testInput
let testStart = startPos testInput
let testEnd = endPos testInput

testStart |> shouldEqual (0,0)
testEnd |> shouldEqual (2,5)

let walked = walkPaths testGrid testStart testEnd
let shortest =
    walked
    |> List.minBy Set.count

shortest.Count |> shouldEqual 31

Installed Packages FsUnit, 5.1.0

In [9]:
open System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_12.txt")
let lines = 
    File.ReadAllLines(sourcePath)

let grid = parseGrid lines
let start = startPos lines
let ends = endPos lines

let walked = walkPaths grid start ends
let shortest =
    walked
    |> List.minBy Set.count

let result = shortest.Count

Error: System.OperationCanceledException: Command :SubmitCode: open System.IO

let sourcePath = Path.Combine(__ ... cancelled.

In [ ]:
printfn "Shortest path %d steps" result